In [31]:
import os
import mido
from copy import deepcopy
import pretty_midi
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("dark_background")

In [36]:
def weighted_average_note_per_beat(midi_data, tempo, beats):
    beat_duration = 60.0 / tempo
    results = []

    for beat in range(beats):
        start_time = beat * beat_duration
        end_time = start_time + beat_duration
        notes = []

        for instrument in midi_data.instruments:
            for note in instrument.notes:
                note_start = note.start
                note_end = note.end
                if note_start < end_time and note_end > start_time:
                    overlap_start = max(note_start, start_time)
                    overlap_end = min(note_end, end_time)
                    overlap_duration = overlap_end - overlap_start
                    if overlap_duration > 0:
                        notes.append((note.pitch, note.velocity, overlap_duration))

        if notes:
            pitches = np.array([note[0] for note in notes])
            velocities = np.array([note[1] for note in notes])
            durations = np.array([note[2] for note in notes])
            normalized_velocities = velocities / np.max(velocities)
            normalized_durations = durations / np.max(durations)

            weighted_avg = np.sum(
                pitches * normalized_velocities * normalized_durations
            ) / np.sum(normalized_velocities * normalized_durations)
            results.append((np.min(pitches), weighted_avg, np.max(pitches)))
        else:
            results.append((0, 0, 0))  # Default to (0, 0, 0) if no notes are present

    return results


def plot_piano_roll(midi_file_path, tempo, beats):
    fs = 100  # frames per second
    midi_data = pretty_midi.PrettyMIDI(midi_file_path)
    piano_roll = midi_data.get_piano_roll(fs)
    beat_frames = np.arange(beats) * (60 / tempo * fs)

    note_tuples = weighted_average_note_per_beat(midi_data, tempo, beats)
    print(note_tuples)

    plt.figure(figsize=(12, 6))
    plt.imshow(piano_roll, aspect="auto", origin="lower", cmap="magma", alpha=0.5)
    plt.colorbar(label="Velocity")
    plt.xlabel("Time (in frames)")
    plt.ylabel("Pitch")

    # Plot average notes as horizontal lines
    for i, beat in enumerate(beat_frames):
        for line in note_tuples[i]:
            plt.hlines(
                y=line,
                xmin=beat,
                xmax=beat + (60 / tempo * fs),
                color="green",
                linewidth=1,
                alpha=0.7,
            )

    plt.title("Piano Roll with Weighted Average Note per Beat")
    plt.show()


# Specify the MIDI file, tempo, and number of beats
midi_file_path = "tmp/playlist/20231220-80-03_0560-0568.mid"
tempo = 120  # BPM
beats = 16  # Number of beats to calculate
plot_piano_roll(midi_file_path, tempo, beats)

TypeError: tuple indices must be integers or slices, not list

In [29]:
def shift_notes_and_save(midi_path: str, n: int, b: float):
    """
    Creates all permutations of a MIDI file by cyclically shifting the notes.

    Args:
        midi_path (str): Path to the MIDI file.
        n (int): Total number of beats in the MIDI file.
        b (float): Number of beats to shift by in each permutation.

    Returns:
        List[mido.MidiFile]: A list of MIDI files with shifted notes.
    """
    # Load the MIDI file
    original_midi = mido.MidiFile(midi_path)
    ticks_per_beat = original_midi.ticks_per_beat
    shift_ticks = int(ticks_per_beat * b)  # Convert beats to ticks for the shift

    # Calculate the number of permutations
    num_permutations = int(n / b)

    permutations = []
    for shift_index in range(num_permutations):
        # Deep copy the original MIDI to avoid altering it
        new_midi = deepcopy(original_midi)
        for track in new_midi.tracks:
            for msg in track:
                if not msg.is_meta and hasattr(msg, "time"):
                    # Adjust the time by the shift amount
                    msg.time = (msg.time + shift_ticks) % (ticks_per_beat * n)

        # Append the new MIDI file to the list
        permutations.append(new_midi)

    return permutations

In [30]:
# Example usage
midi_permutations = shift_notes_and_save("path_to_midi_file.mid", 16, 1)
for i, midi_file in enumerate(midi_permutations):
    midi_file.save(f"output_shifted_{i}.mid")

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_midi_file.mid'

In [ ]:
rec_path = os.path.join("tmp", "recordings")

for recording in os.listdir(rec_path):
    print(f"playing {recording}")
    midi = mido.MidiFile(os.path.join(rec_path, recording))
    midi.print_tracks()

    with mido.open_output("Disklavier") as outport:
        for msg in midi.play():
            outport.send(msg)

playing recording-040-240501_143824.mid
=== Track 0
MetaMessage('set_tempo', tempo=1500000, time=0)
Message('note_on', channel=0, note=48, velocity=66, time=0)
Message('note_off', channel=0, note=48, velocity=53, time=72)
Message('note_on', channel=0, note=49, velocity=66, time=293)
Message('note_off', channel=0, note=49, velocity=52, time=79)
Message('note_on', channel=0, note=50, velocity=61, time=257)
Message('note_off', channel=0, note=50, velocity=54, time=101)
Message('note_on', channel=0, note=51, velocity=70, time=259)
Message('note_off', channel=0, note=51, velocity=51, time=65)
Message('note_on', channel=0, note=52, velocity=70, time=299)
Message('note_off', channel=0, note=52, velocity=56, time=103)
Message('note_on', channel=0, note=53, velocity=64, time=259)
Message('note_off', channel=0, note=53, velocity=53, time=79)
Message('note_on', channel=0, note=54, velocity=62, time=265)
Message('note_off', channel=0, note=54, velocity=58, time=67)
Message('note_on', channel=0, no

In [ ]:
mbt = mido.bpm2tempo(1500000)
mspb = lambda tempo: int((60 / tempo) * 1_000_000)
print(mspb(40))

1500000
